[Data Cleaning](#Data-Cleaning)<br>
&emsp;[1. Import Libraries](#1.-Import-Libraries)<br>
&emsp;[2. Reading Dataset](#2.-Reading-Dataset)<br>
&emsp;[3. Preliminary Analysis](#3.-Preliminary-Analysis)<br>
&emsp;&emsp;[3.1 Check Data Types](#3.1-Check-Data-Types)<br>
&emsp;&emsp;[3.2 Check for Duplicates](#3.2-Check-for-Duplicates)<br>
&emsp;&emsp;[3.3 Check for Missing Values](#3.3-Check-for-Missing-Values)<br>
&emsp;[4. Detailed Analysis](#4.-Detailed-Analysis)<br>
&emsp;&emsp;[4.1 Airline](#4.1-Airline)<br>
&emsp;&emsp;[4.2 Date of Journey](#4.2-Date-of-Journey)<br>
&emsp;&emsp;[4.3 Source](#4.3-Source)<br>
&emsp;&emsp;[4.4 Destination](#4.4-Destination)<br>
&emsp;&emsp;[4.5 Route](#4.5-Route)<br>
&emsp;&emsp;[4.6 Departure Time](#4.6-Departure-Time)<br>
&emsp;&emsp;[4.7 Arrival Time](#4.7-Arrival-Time)<br>
&emsp;&emsp;[4.8 Duration](#4.8-Duration)<br>
&emsp;&emsp;[4.9 Total Stops](#4.9-Total-Stops)<br>
&emsp;&emsp;[4.10 Additional Info](#4.10-Additional-Info)<br>
&emsp;&emsp;[4.11 Price](#4.11-Price)<br>
&emsp;[5. Cleaning Operation](#5.-Cleaning-Operation)<br>
&emsp;[6. Split The Datset](#6.-Split-The-Datset)<br>
&emsp;[7. Export The Datset Subset](#7.-Export-The-Datset-Subset)<br>

## 1. Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2. Reading Dataset

In [2]:
PROJECT_DIR = 'R:\Jaydeep\Flight-Price-Prediction'
DATA_DIR = 'data'
MAIN_DATASET_NAME = 'flight_price'

In [3]:
def get_dataset(dataset_name):
    file_name = f'{dataset_name}.csv'
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [4]:
df = get_dataset(MAIN_DATASET_NAME)

In [5]:
df.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9521,Multiple carriers,12/06/2019,Delhi,Cochin,DEL → BOM → COK,12:50,21:00,8h 10m,1 stop,No info,7005
7998,Air Asia,18/05/2019,Delhi,Cochin,DEL → BLR → COK,07:55,13:25,5h 30m,1 stop,No info,8759
4632,Air India,6/03/2019,Delhi,Cochin,DEL → JAI → BOM → COK,12:00,19:15,7h 15m,2 stops,No info,11858
2808,IndiGo,15/03/2019,Mumbai,Hyderabad,BOM → HYD,06:20,07:45,1h 25m,non-stop,No info,3342
8226,Multiple carriers,21/03/2019,Delhi,Cochin,DEL → BOM → COK,04:45,18:50,14h 5m,1 stop,No info,11098


## 3. Preliminary Analysis

### 3.1 Check Data Types

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### 3.2 Check for Duplicates

In [7]:
df.duplicated().sum()

220

### 3.3 Check for Missing Values

In [8]:
df.isnull().sum().sum()

2

In [9]:
df.isnull().sum()[lambda x: x>0]

Route          1
Total_Stops    1
dtype: int64

In [10]:
df.loc[df['Route'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [11]:
df.loc[df['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


- There are 10683 rows & 11 columns.
- Need to convert `Date_of_Journey` `Dep_Time` & `Arrival_Time` to datetime.
- Need to convert `Duration` & `Total_Stops` to numeric.
- There are 220 duplicates, it should be removed.
- There are 2 null values.
    - `Route`:1
    - `Total_Stops`:1

## 4. Detailed Analysis

In [12]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

### 4.1 Airline

In [13]:
df.Airline.nunique()

12

In [14]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [15]:
(
	df
	.Airline
	.str.replace(" Premium economy", "")
	.str.replace(" Business", "")
	.str.title()
).sample(5)

1587          Jet Airways
8144              Vistara
4859            Air India
7337          Jet Airways
3470    Multiple Carriers
Name: Airline, dtype: object

### 4.2 Date of Journey

In [16]:
df.Date_of_Journey.nunique()

44

In [17]:
df.Date_of_Journey.unique()

array(['24/03/2019', '1/05/2019', '9/06/2019', '12/05/2019', '01/03/2019',
       '24/06/2019', '12/03/2019', '27/05/2019', '1/06/2019',
       '18/04/2019', '9/05/2019', '24/04/2019', '3/03/2019', '15/04/2019',
       '12/06/2019', '6/03/2019', '21/03/2019', '3/04/2019', '6/05/2019',
       '15/05/2019', '18/06/2019', '15/06/2019', '6/04/2019',
       '18/05/2019', '27/06/2019', '21/05/2019', '06/03/2019',
       '3/06/2019', '15/03/2019', '3/05/2019', '9/03/2019', '6/06/2019',
       '24/05/2019', '09/03/2019', '1/04/2019', '21/04/2019',
       '21/06/2019', '27/03/2019', '18/03/2019', '12/04/2019',
       '9/04/2019', '1/03/2019', '03/03/2019', '27/04/2019'], dtype=object)

In [18]:
pd.to_datetime(df.Date_of_Journey, dayfirst=True).sample(5)

6750   2019-06-09
8666   2019-06-15
365    2019-05-15
4330   2019-03-24
5771   2019-03-18
Name: Date_of_Journey, dtype: datetime64[ns]

### 4.3 Source

In [19]:
df.Source.nunique()

5

In [20]:
df.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### 4.4 Destination

In [21]:
df.Destination.nunique()

6

In [22]:
df.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

### 4.5 Route

In [23]:
df.Route.sample(5)

3135                 BLR → DEL
9452     CCU → GAU → DEL → BLR
9019           BLR → BOM → DEL
10200    CCU → IXR → DEL → BLR
7673           DEL → BOM → COK
Name: Route, dtype: object

### 4.6 Departure Time

In [24]:
df.Dep_Time.nunique()

222

In [25]:
df.Dep_Time.sample(5)

7282    23:55
7167    14:05
2504    20:55
5491    17:00
2494    19:15
Name: Dep_Time, dtype: object

In [26]:
(
    df
    .Dep_Time
    .loc[lambda x: x.str.contains('[^0-9:]')]
)

Series([], Name: Dep_Time, dtype: object)

In [27]:
pd.to_datetime(df.Dep_Time).dt.time.sample(5)

C:\Users\rohan\AppData\Local\Temp\ipykernel_22156\2859816643.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df.Dep_Time).dt.time.sample(5)


9635    15:05:00
7456    05:05:00
9215    16:50:00
1730    06:40:00
3115    11:40:00
Name: Dep_Time, dtype: object

### 4.7 Arrival Time

In [28]:
df.Arrival_Time.sample(5)

4748    18:10 25 Mar
573            22:35
684     00:15 28 May
2006           22:40
4655    19:15 28 Jun
Name: Arrival_Time, dtype: object

In [29]:
(
    df
    .Arrival_Time
    .loc[lambda x: x.str.contains('[^0-9:]')]
    .str.split(n=1)
    .str.get(1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

### 4.8 Duration

In [30]:
df.Duration.sample(5)

4917     6h 20m
3340    21h 25m
5907     2h 50m
8519    12h 25m
7601     6h 15m
Name: Duration, dtype: object

In [31]:
(
    df
    .Duration
    .loc[lambda x: ~x.str.contains('m')]
    .unique()
)

array(['19h', '23h', '22h', '12h', '3h', '5h', '10h', '18h', '24h', '15h',
       '16h', '8h', '14h', '20h', '13h', '11h', '9h', '27h', '26h', '4h',
       '7h', '30h', '21h', '28h', '47h', '6h', '25h', '38h', '34h'],
      dtype=object)

In [32]:
(
    df
    .Duration
    .loc[lambda x: ~x.str.contains('h')]
    .unique()
)

array(['5m'], dtype=object)

In [33]:
(
    df
    .Duration
    .loc[lambda x: ~x.str.contains('h')]
)

6474    5m
Name: Duration, dtype: object

In [34]:
df.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [35]:
(
    df
    .Duration
    .drop(index=6474)
    .str.split(expand=True)
    .set_axis(['hour', 'minute'], axis=1)
    .assign(
        hour = lambda df_: (
            df_
            .hour
            .str.replace('h','')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df_: (
            df_
            .minute
            .str.replace('m','')
            .fillna('0')
            .astype(int)
        )
    )
    .sum(axis=1)
    .rename('duration_minutes')
    .to_frame()
    .join(df.Duration)
).sample(5)

,duration_minutes,Duration
2427,590,9h 50m
3398,155,2h 35m
10244,475,7h 55m
7465,180,3h
4107,170,2h 50m


### 4.9 Total Stops

In [36]:
df.Total_Stops.nunique()

5

In [37]:
df.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [38]:
(
    df
    .Total_Stops
    .str.replace('non-stop', '0')
    .str.replace('stops?', '', regex=True)
    .pipe(lambda x: pd.to_numeric(x))
).unique()

array([ 0.,  2.,  1.,  3., nan,  4.])

### 4.10 Additional Info

In [39]:
df.Additional_Info.nunique()

10

In [40]:
df.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [41]:
(
    df
    .Additional_Info
    .str.lower()
).unique()

array(['no info', 'in-flight meal not included',
       'no check-in baggage included', '1 short layover',
       '1 long layover', 'change airports', 'business class',
       'red-eye flight', '2 long layover'], dtype=object)

### 4.11 Price

In [42]:
df.Price.sample(5)

9479    15119
6156     3841
2226     5583
7132     5117
3597    11228
Name: Price, dtype: int64

In [43]:
df.Price.nunique()

1870

## 5. Cleaning Operation

In [44]:
def clean_column_names(df):
    # Convert all column names to lowercase for consistency
    return df.rename(columns=str.lower)

def strip_string_columns(df):
    # Identify columns with string data type
    string_columns = df.select_dtypes(include='O').columns
    # Strip leading and trailing whitespace from all string columns
    for col in string_columns:
        df[col] = df[col].str.strip()
    return df

def clean_airline_names(df):
    # Clean and standardize 'airline' column by removing specific substrings and title-casing
    df['airline'] = (
        df['airline']
        .str.replace(" Premium economy", "")
        .str.replace(" Business", "")
        .str.title()
    )
    return df

def convert_dates(df):
    # Convert 'date_of_journey' column to datetime format, assuming day-first format
    df['date_of_journey'] = pd.to_datetime(df['date_of_journey'], dayfirst=True)
    return df

def convert_times(df):
    # Convert 'dep_time' and 'arrival_time' columns to time format
    df['dep_time'] = pd.to_datetime(df['dep_time']).dt.time
    df['arrival_time'] = pd.to_datetime(df['arrival_time']).dt.time
    return df

def convert_duration(df):
    # Split 'duration' column into hours and minutes
    duration_split = df['duration'].str.split(" ", expand=True).set_axis(["hour", "minute"], axis=1)
    # Convert hours to minutes and fill missing minutes with 0
    duration_split['hour'] = duration_split['hour'].str.replace("h", "").astype(int).mul(60)
    duration_split['minute'] = duration_split['minute'].str.replace("m", "").fillna("0").astype(int)
    # Sum hours and minutes to get total duration in minutes
    df['duration_minute'] = duration_split.sum(axis=1)
    # drop duration column
    df = df.drop(columns=['duration'])
    return df

def convert_total_stops(df):
    # Standardize and convert 'total_stops' column to numeric
    df['total_stops'] = (
        df['total_stops']
        .replace("non-stop", "0")  # Replace 'non-stop' with '0'
        .str.replace(" stops?", "", regex=True)  # Remove ' stop' or ' stops'
        .pipe(lambda ser: pd.to_numeric(ser))  # Convert to numeric
    )
    return df

def lower_additional_info(df):
    # Convert 'additional_info' column to lowercase
    df['additional_info'] = df['additional_info'].str.lower()
    return df

def preprocess_df(df):
    # Drop rows where 'Duration' is '5m' and drop the 'Route' column
    df = df.drop(index=df[df['Duration'].isin(['5m'])].index, columns=['Route'])
    # Drop duplicate rows to ensure data consistency
    df = df.drop_duplicates()
    # Drop null values
    df = df.dropna()
    # Apply all preprocessing steps in sequence to clean and standardize the DataFrame
    df = clean_column_names(df)         # Convert column names to lowercase
    df = strip_string_columns(df)       # Strip leading/trailing whitespace from string columns
    df = clean_airline_names(df)        # Clean and standardize 'airline' column
    df = convert_dates(df)              # Convert 'date_of_journey' column to datetime format
    df = convert_times(df)              # Convert 'dep_time' and 'arrival_time' columns to time format
    df = convert_duration(df)           # Convert 'duration' column to total minutes
    df = convert_total_stops(df)        # Standardize and convert 'total_stops' column to numeric
    df = lower_additional_info(df)      # Convert 'additional_info' column to lowercase
    # Drop duplicate rows to ensure data consistency
    df = df.drop_duplicates()
    return df

In [45]:
df = preprocess_df(df)

C:\Users\rohan\AppData\Local\Temp\ipykernel_22156\480543875.py:30: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['dep_time'] = pd.to_datetime(df['dep_time']).dt.time
C:\Users\rohan\AppData\Local\Temp\ipykernel_22156\480543875.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['arrival_time'] = pd.to_datetime(df['arrival_time']).dt.time


In [46]:
columns = [
    'airline', 'date_of_journey', 'source', 'destination', 'dep_time',
    'arrival_time',  'duration_minute', 'total_stops', 'additional_info',
    'price',
]

In [47]:
df = df[columns]

In [48]:
df.sample(10)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration_minute,total_stops,additional_info,price
2787,Jet Airways,2019-06-06,Kolkata,Banglore,18:55:00,04:40:00,585,1,in-flight meal not included,7757
1860,Jet Airways,2019-05-09,Kolkata,Banglore,16:30:00,08:15:00,945,1,no info,13941
8758,Air India,2019-05-06,Kolkata,Banglore,12:00:00,18:30:00,1830,2,no info,10151
5223,Spicejet,2019-06-01,Kolkata,Banglore,14:55:00,17:25:00,150,0,no check-in baggage included,3841
1194,Air India,2019-03-21,Banglore,New Delhi,11:50:00,23:55:00,725,1,no info,5092
8210,Jet Airways,2019-05-01,Kolkata,Banglore,21:10:00,09:20:00,730,1,no info,14781
2673,Jet Airways,2019-04-01,Kolkata,Banglore,06:30:00,12:00:00,330,1,in-flight meal not included,10056
2842,Air India,2019-05-01,Delhi,Cochin,20:40:00,09:25:00,765,1,no info,7480
6822,Jet Airways,2019-04-01,Kolkata,Banglore,20:00:00,19:50:00,1430,1,in-flight meal not included,7064
6942,Indigo,2019-05-12,Chennai,Kolkata,19:35:00,21:55:00,140,0,no info,3597


## 6. Split The Datset

In [49]:
X = df.drop(columns="price")
y = df.price.copy()

X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(6693, 9) (6693,)
(1674, 9) (1674,)
(2092, 9) (2092,)


## 7. Export The Datset Subset

In [50]:
def export_dataset(X, y, name):
	file_name = f"{name}.csv"
	file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
	X.join(y).to_csv(file_path, index=False)

In [51]:
export_dataset(X_train, y_train, 'train')
export_dataset(X_val, y_val, 'validation')
export_dataset(X_test, y_test, 'test')

In [52]:
get_dataset('train').sample(3)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration_minute,total_stops,additional_info,price
4202,Air India,2019-03-21,Banglore,New Delhi,08:15:00,11:20:00,1625,2,no info,7642
5132,Indigo,2019-05-21,Kolkata,Banglore,17:15:00,19:50:00,155,0,no info,4804
3536,Multiple Carriers,2019-03-27,Delhi,Cochin,10:20:00,21:00:00,640,1,no info,6637


In [53]:
get_dataset('validation').sample(3)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration_minute,total_stops,additional_info,price
332,Air India,2019-04-01,Kolkata,Banglore,17:30:00,08:55:00,925,1,no info,7893
1622,Spicejet,2019-04-15,Kolkata,Banglore,06:55:00,09:30:00,155,0,no info,3873
533,Indigo,2019-06-27,Mumbai,Hyderabad,16:55:00,18:25:00,90,0,no info,3175


In [54]:
get_dataset('test').sample(3)

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration_minute,total_stops,additional_info,price
464,Jet Airways,2019-03-27,Delhi,Cochin,13:00:00,04:25:00,925,1,in-flight meal not included,6960
1036,Jet Airways,2019-06-12,Delhi,Cochin,08:00:00,19:00:00,660,1,in-flight meal not included,10577
1087,Multiple Carriers,2019-03-06,Delhi,Cochin,09:45:00,21:20:00,695,2,no info,10444


In [55]:
# (
#     df
#     .drop(index=df[df.Duration.isin(['5m'])].index, columns=['Route'])
#     .drop_duplicates()
#     .rename(columns=str.lower)
#     .assign(
#         **{
#             col: df[col].str.strip()
#             for col in df.select_dtypes(include="O").columns
#         }
#     )
#     .assign(
#         airline = lambda df_: (
# 				df_
# 				.airline
# 				.str.replace(" Premium economy", "")
# 				.str.replace(" Business", "")
# 				.str.title()
# 		),
#         date_of_journey = lambda df_: (
#             pd.to_datetime(df_.date_of_journey, dayfirst=True)
#         ),
#         dep_time = lambda df_: (
#                 pd.to_datetime(df_.dep_time).dt.time
#         ),
#         arrival_time = lambda df_: (
#             pd.to_datetime(df_.arrival_time).dt.time
#         ),
#         duration = lambda df_: (
#             df_.duration.pipe(lambda x: (
#                 x
#                 .str.split(" ", expand=True)
#         		.set_axis(["hour", "minute"], axis=1)
#         		.assign(
#                     hour=lambda df_: (
#                         df_
#         				.hour
#         				.str.replace("h", "")
#         				.astype(int)
#         				.mul(60)
#             		),
#         			minute=lambda df_: (
#         				df_
#         				.minute
#         				.str.replace("m", "")
#         				.fillna("0")
#         				.astype(int)
#         			)
#         		).sum(axis=1)
#             )
#           )
#         ),
#         total_stops = lambda df_: (
# 				df_
# 				.total_stops
# 				.replace("non-stop", "0")
# 				.str.replace(" stops?", "", regex=True)
# 				.pipe(lambda ser: pd.to_numeric(ser))
# 		),
#         additional_info = lambda df_ : (
#             df_
#             .additional_info
#             .str
#             .lower()
#         )
#     )
# ).sample(10)